In [1]:
################################################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedGroupKFold
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping


from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier

from collections import defaultdict
import visualkeras

In [2]:
###############################################################################################
# Čitanje podataka; formiranje baze koja se sastoji od rečnika sa EKG, PPG, PCG i ACC signalima
###############################################################################################

ppg_data = pd.read_csv('ppg_beats_for_classification_fixed_length_20_hz.csv')
ppg_data = ppg_data[~np.isnan(ppg_data.iloc[:,-1])]

y = ppg_data.iloc[:,-1].to_numpy() # označene labele
y_bin = (y != 0).astype(int) # binarne labele
subjects = ppg_data.iloc[:,-2] # ovde imam sve ispitanike 

ppg_data = ppg_data.iloc[:,:-2].to_numpy()

ecg_data = pd.read_csv('ecg_beats_for_classification_fixed_length_500_hz.csv')
ecg_data= ecg_data[~np.isnan(ecg_data.iloc[:,-1])].iloc[:,:-2].to_numpy()

pcg_data = pd.read_csv('pcg_beats_for_classification_fixed_length_200_hz.csv')
pcg_data= pcg_data[~np.isnan(pcg_data.iloc[:,-1])].iloc[:,:-2].to_numpy()

acc_data = pd.read_csv('acc_beats_for_classification_fixed_length_50_hz.csv')
acc_data= acc_data[~np.isnan(acc_data.iloc[:,-1])].iloc[:,:-2].to_numpy()

# Učešljavanje podataka o svim signalima
X = []
for i in range(0, np.shape(ecg_data)[0]):
    signal = {
        'ecg': ecg_data[i].reshape(-1,1),
        'ppg': ppg_data[i].reshape(-1,1),
        'pcg': pcg_data[i].reshape(-1,1),
        'acc': acc_data[i].reshape(-1,1),
        'subject': subjects.iloc[i]
    }
    X.append(signal)


In [3]:
# Mešanje podataka iz skupa (obučavajućeg)
def shuffle_data(X, y):
    assert len(X) == len(y)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    return X[indices], y[indices]

## Multimodal CNN LSTM neural network

In [4]:
# Model neuralne mreže koji ćemo koristiti

def branch(input_shape):
    input_layer = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv1D(16, kernel_size=5, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(input_layer)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    # Block 2
    x = layers.Conv1D(16, kernel_size=5, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    # Block 3
    x = layers.Conv1D(16, kernel_size=5, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    #x = layers.GlobalAveragePooling1D()(x)

    # LSTM block
    x = layers.LSTM(32, return_sequences=False, dropout=0.3, recurrent_dropout=0.3)(x)
    x = layers.BatchNormalization()(x)

    return models.Model(inputs=input_layer, outputs=x)
    
def build_multimodal_model(ecg_shape, ppg_shape, pcg_shape, acc_shape, num_classes):
    #ecg_input = layers.Input(shape=ecg_shape)
    ppg_input = layers.Input(shape=ppg_shape)
    pcg_input = layers.Input(shape=pcg_shape)
    acc_input = layers.Input(shape=acc_shape)

    #ecg_branch = branch(ecg_shape)(ecg_input)
    ppg_branch = branch(ppg_shape)(ppg_input)
    pcg_branch = branch(pcg_shape)(pcg_input)
    acc_branch = branch(acc_shape)(acc_input)

    #x = layers.Concatenate()([ecg_branch, ppg_branch, pcg_branch, acc_branch])
    x = layers.Concatenate()([ppg_branch, pcg_branch, acc_branch])
    print(np.shape(x))
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=[ppg_input, pcg_input, acc_input], outputs=outputs)
    model.compile(optimizer=Adam(weight_decay=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [5]:
modalities = ['ecg', 'ppg', 'pcg', 'acc']

def split_to_modalities(data):
    #ecg_data = np.stack([sample['ecg'] for sample in data])
    ppg_data = np.stack([sample['ppg'] for sample in data])
    pcg_data = np.stack([sample['pcg'] for sample in data])
    acc_data = np.stack([sample['acc'] for sample in data])
    return [ppg_data, pcg_data, acc_data]

def train_val_split(X_train_val, y_train_val, groups_train_val, outer_fold):
    # Use StratifiedGroupKFold again for train/val split
    inner_kf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=outer_fold + 42)

    for inner_train_idx, inner_val_idx in inner_kf.split(X_train_val, y_train_val, groups_train_val):
        X_train, y_train = X_train_val[inner_train_idx], y_train_val[inner_train_idx]
        X_val, y_val = X_train_val[inner_val_idx], y_train_val[inner_val_idx]
        break  # Just take the first split

    return X_train, y_train, X_val, y_val

def standardize_data(X_train, X_val, X_test):
    means = {mod: [] for mod in modalities}
    stds  = {mod: [] for mod in modalities}

    for sample in X_train:
        for mod in modalities:
            signal = sample[mod].flatten()
            means[mod].append(signal)
            stds[mod].append(signal)

    for mod in modalities:
        all_values = np.concatenate(means[mod])
        means[mod] = np.mean(all_values)
        stds[mod]  = np.std(all_values)

    for sample in X_train:
        for mod in modalities:
            sample[mod] = (sample[mod] - means[mod]) / (stds[mod] + 1e-8)
    for sample in X_test:
        for mod in modalities:
            sample[mod] = (sample[mod] - means[mod]) / (stds[mod] + 1e-8)
    for sample in X_val:
        for mod in modalities:
            sample[mod] = (sample[mod] - means[mod]) / (stds[mod] + 1e-8)

    return X_train, X_test, X_val
    
    

In [6]:
def accuracy_estimation(model, X, y):
    
    outputs = model.predict(split_to_modalities(X))
    sigmoids = defaultdict(list)
    predictions = defaultdict(float)
    true_labels = defaultdict(float)
    counter_true = 0
    counter = 0
    for i in range(0, np.shape(X)[0]):
        sigmoids[X[i]['subject']].append(outputs[i])
        true_labels[X[i]['subject']] = y[i]
    for subj in sigmoids:
        counter = counter + 1
        predictions[subj] = np.round(np.mean(sigmoids[subj]))
        if predictions[subj] == true_labels[subj]:
            counter_true = counter_true + 1
    
    return counter_true/counter

def accuracy_estimation_multiclass(model, X, y):

    outputs = model.predict(split_to_modalities(X))
    softmaxes = defaultdict(list)
    predictions = defaultdict(float)
    true_labels = defaultdict(float)
    counter_true = 0
    counter = 0
    for i in range(0, np.shape(X)[0]):
        softmaxes[X[i]['subject']].append(np.argmax(outputs[i], axis=1))
        true_labels[X[i]['subject']] = y[i]
    for subj in softmaxes:
        counter = counter + 1
        predictions[subj] = np.argmax(np.mean(np.array(softmaxes[subj])))
        if predictions[subj] == true_labels[subj]:
            counter_true = counter_true + 1
    
    return counter_true/counter

def accuracy_estimation_multiclass(model, criterion, X, y):

    outputs = fusion_model.model_output(model, criterion, X, y)
    outputs = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
    print(outputs)
    softmaxes = defaultdict(list)
    predictions = defaultdict(float)
    true_labels = defaultdict(float)
    counter_true = 0
    counter = 0
    for i in range(0, np.shape(groups)[0]):
        softmaxes[groups.iloc[i]].append(outputs[i])
        true_labels[groups.iloc[i]] = y[i]
    for subj in softmaxes:
        counter = counter + 1
        classes, counts = torch.unique(torch.tensor(softmaxes[subj]), return_counts=True)
        predictions[subj] = classes[torch.argmax(counts)]
        if predictions[subj] == true_labels[subj]:
            counter_true = counter_true + 1
    
    return counter_true/counter

In [7]:
ecg_shape = (417, 1)
ppg_shape = (17, 1)
pcg_shape = (167,1)
acc_shape = (42,1)
num_classes = 4
class_names = ["class0", "class1"]

X = np.array(X)
y = np.array(y_bin)
groups = np.array(subjects)

outer_kf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
estimated_accuracies = []
precisions = []
recalls = []
f1scores = []
all_reports = {cls: {'precision': [], 'recall': [], 'f1-score': []} for cls in class_names}

for outer_fold, (train_val_idx, test_idx) in enumerate(outer_kf.split(X, y, groups)):
    print(f"\n=== Outer Fold {outer_fold+1} ===")
    
    X_train_val, X_test = X[train_val_idx], X[test_idx]
    y_train_val, y_test = y[train_val_idx], y[test_idx]
    groups_train_val = groups[train_val_idx]

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    X_train, y_train, X_val, y_val = train_val_split(X_train_val, y_train_val, groups_train_val, outer_fold)
    X_train, X_test, X_val = standardize_data(X_train, X_val, X_test)

    X_train, y_train = shuffle_data(X_train, y_train)
    X_val, y_val = shuffle_data(X_val, y_val)
    
    model = build_multimodal_model(ecg_shape, ppg_shape, pcg_shape, acc_shape, num_classes)

    # Early stopping to avoid overfitting
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train
    model.fit(
        split_to_modalities(X_train), y_train,
        validation_data=(split_to_modalities(X_val), y_val),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop]
    )
    
    # Evaluate
    loss, acc = model.evaluate(split_to_modalities(X_test), y_test)
    print(np.unique(y_test))
    y_pred = np.round(model.predict(split_to_modalities(X_test)))
    print(np.unique(y_pred))
    report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True, zero_division=0)
    for cls in class_names:
        for metric in ['precision', 'recall', 'f1-score']:
            all_reports[cls][metric].append(report[cls][metric])

    accuracies.append(acc)
    print(f"Test Accuracy standard: {acc: .4f}")
    acc = accuracy_estimation(model, X_test, y_test)
    estimated_accuracies.append(acc)
    print(f"Test Accuracy: {acc:.4f}")
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))
    f1scores.append(f1_score(y_test, y_pred))
    

print(f"Mean test accuracy: {np.mean(accuracies):.4f}, std: {np.std(accuracies):.4f}")
print(f"Mean test accuracy: {np.mean(estimated_accuracies):.4f}, std: {np.std(estimated_accuracies):.4f}")
print(f"Mean test precision: {np.mean(precisions):.4f}, std: {np.std(precisions):.4f}")
print(f"Mean test recall: {np.mean(recalls):.4f}, std: {np.std(recalls):.4f}")
print(f"Mean test F1 score: {np.mean(f1scores):.4f}, std: {np.std(f1scores):.4f}")




=== Outer Fold 1 ===
(None, 96)
Epoch 1/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.6206 - loss: 0.7369 - val_accuracy: 0.6335 - val_loss: 0.9194
Epoch 2/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7370 - loss: 0.5343 - val_accuracy: 0.6003 - val_loss: 1.1282
Epoch 3/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7696 - loss: 0.4683 - val_accuracy: 0.6822 - val_loss: 0.9686
Epoch 4/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8059 - loss: 0.3997 - val_accuracy: 0.5925 - val_loss: 1.3854
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8581 - loss: 0.3375
[0 1]
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
[0. 1.]
Test Accuracy standard:  0.6366
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Test Accuracy: 0.6667

=== Outer Fold 2 ===
(None, 96)
Epoch 1/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.6006 - loss: 0.7205 - val_accuracy: 0.7885 - val_loss: 0.5554
Epoch 2/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.778

In [8]:
summary = pd.DataFrame(index=class_names, columns=['Precision (mean ± std)', 'Recall (mean ± std)', 'F1 (mean ± std)'])

for cls in class_names:
    precision = np.array(all_reports[cls]['precision'])
    recall = np.array(all_reports[cls]['recall'])
    f1 = np.array(all_reports[cls]['f1-score'])

    summary.loc[cls, 'Precision (mean ± std)'] = f"{precision.mean():.2f} ± {precision.std():.2f}"
    summary.loc[cls, 'Recall (mean ± std)'] = f"{recall.mean():.2f} ± {recall.std():.2f}"
    summary.loc[cls, 'F1 (mean ± std)'] = f"{f1.mean():.2f} ± {f1.std():.2f}"

print(summary)
print(model.summary())

       Precision (mean ± std) Recall (mean ± std) F1 (mean ± std)
class0            0.61 ± 0.15         0.68 ± 0.18     0.61 ± 0.08
class1            0.72 ± 0.10         0.61 ± 0.21     0.63 ± 0.11


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 17, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_25      │ (None, 167, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_26      │ (None, 42, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_16       │ (None, 32)        │      9,280 │ input_layer_24[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_17       │ (None, 32)        │      9,280 │ input_layer_25[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_18       │ (None, 32)        │      9,280 │ input_layer_26[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 96)        │          0 │ functional_16[0]… │
│ (Concatenate)       │                   │            │ functional_17[0]… │
│                     │                   │            │ functional_18[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      3,104 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_49          │ (None, 32)        │          0 │ dense_8[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │         33 │ dropout_49[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 91,973 (359.27 KB)

 Trainable params: 30,497 (119.13 KB)

 Non-trainable params: 480 (1.88 KB)

 Optimizer params: 60,996 (238.27 KB)

None
